In [130]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community faiss-cpu

In [131]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/Library/Python/3.9/site-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [132]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [133]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter

loader = WebBaseLoader(urls)

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

Created a chunk of size 2731, which is longer than the specified 1000
Created a chunk of size 1538, which is longer than the specified 1000
Created a chunk of size 1380, which is longer than the specified 1000
Created a chunk of size 2352, which is longer than the specified 1000
Created a chunk of size 1953, which is longer than the specified 1000
Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 1475, which is longer than the specified 1000
Created a chunk of size 2881, which is longer than the specified 1000
Created a chunk of size 1980, which is longer than the specified 1000
Created a chunk of size 4145, which is longer than the specified 1000
Created a chunk of size 2528, which is longer than the specified 1000
Created a chunk of size 2158, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 1112, which is longer than the specified 1000
Created a chunk of s

In [134]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(texts, embeddings)

In [135]:
user_query = "agent memory"

vector_store_retrievers = [vectorstore.as_retriever(), vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
), vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
), vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
), vectorstore.as_retriever(search_kwargs={'k': 1}), vectorstore.as_retriever(
    search_kwargs={'filter': {'paper_title': 'GPT-4 Technical Report'}}
)]

In [136]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

parser = JsonOutputParser()

relevant_prompt = PromptTemplate(
    template="Determine if the query is relevant to docs.\n{format_instructions}\n{query}\n{docs}",
    input_variables=["query", "docs"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

relevant_chain = relevant_prompt | llm | parser

relevant_retrievers = []
irrelevant_retrievers = []

for retriever in vector_store_retrievers:
    docs = retriever.invoke(user_query)
    res = relevant_chain.invoke({"query": user_query, "docs": docs})
    print(res)
    if res.get('relevant', False):
        relevant_retrievers.append(retriever)
    else:
        irrelevant_retrievers.append(retriever)

{'relevant': True, 'explanation': "The query 'agent memory' is relevant to the documents as they discuss the concept of LLM-powered autonomous agents, including various components such as memory systems, task decomposition, and the reflection mechanism, all of which are pertinent to the idea of memory in agent systems."}
{'relevant': True, 'reason': "The query 'agent memory' is relevant to the provided documents, particularly the one discussing LLM-powered autonomous agents which includes details about memory systems such as Short-Term Memory (STM), Long-Term Memory (LTM), and the role of memory in agent systems."}
{'relevant': True, 'documents': [{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiali

No relevant docs were retrieved using the relevance score threshold 0.8


{'isRelevant': False, 'message': "The query 'agent memory' does not provide enough context to determine relevance to specific documents."}
{'relevant': True}
{'relevant': False, 'message': "The query 'agent memory' is too vague and does not provide enough context to determine its relevance to specific documents."}


In [139]:
relevant_checker = 'I like an apple'

validated_retrievers = []

for retriever in relevant_retrievers:
    docs = retriever.invoke(relevant_checker)
    res = relevant_chain.invoke({"query": relevant_checker, "docs": docs})
    if not res.get('relevant', True):
        print("Not relevant")
    else:
        print("Relevant")

Not relevant
Not relevant
Not relevant
Not relevant


In [167]:
hallucination_prompt = PromptTemplate(
    template="Determine if the query has hallucination or not.\n{format_instructions}\n{query}\n{docs}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

hallucination_chain = hallucination_prompt | llm | parser

for retriever in relevant_retrievers:
    docs = retriever.invoke(user_query)
    res = hallucination_chain.invoke({"query": user_query, "docs": docs})
    
    if res.get('hallucination', True):
        break
    else:
        print(docs[0])

page_content='Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'de